In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [10]:
df = pd.read_csv('/Users/brandynklee/Python_Projects/fantasy_footballl_22/data/all_compiled.csv', index_col = 0)
comma_col = ['REC_YD', 'RUSH_YD', 'PASS_YD']

for col in comma_col:
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].astype(float)

df['HALF_PPR'] = (df['RUSH_YD'] + df['REC_YD'])*0.1 + df['PASS_YD']*0.04 + (df['RUSH_TD'] + df['REC_TD'])*6 \
    + df['PASS_TD']*4 + df['REC']*.5 + df['FL']*-2 + df['INTS']*-2 

df.sort_values(by = 'HALF_PPR', ascending = False).head(20)


,Player,Team,POS,RUSH_ATT,RUSH_YD,RUSH_TD,REC,REC_YD,REC_TD,FL,PASS_ATT,CMP,PASS_YD,PASS_TD,INTS,HALF_PPR
1,Josh Allen,BUF,QB,119.0,699.2,6.4,0.0,0.0,0.0,3.8,598.0,381.0,4342.7,34.7,15.8,381.628
2,Patrick Mahomes II,KC,QB,66.3,374.1,3.1,0.0,0.0,0.0,3.6,638.2,420.5,4727.0,34.5,13.0,349.890
3,Justin Herbert,LAC,QB,61.8,281.7,3.2,0.0,0.0,0.0,2.0,628.1,416.5,4619.9,34.0,14.2,335.766
4,Jalen Hurts,PHI,QB,143.5,760.6,8.3,0.0,0.0,0.0,3.4,482.2,295.9,3490.8,24.7,13.3,330.892
5,Lamar Jackson,BAL,QB,166.1,947.0,4.0,0.0,0.0,0.0,4.4,524.1,340.7,3705.2,24.2,13.9,327.108
9,Aaron Rodgers,GB,QB,36.3,126.5,2.1,0.0,0.0,0.0,2.1,563.0,377.3,4254.9,34.4,8.0,312.846
8,Dak Prescott,DAL,QB,53.5,189.1,2.6,0.0,0.0,0.0,5.5,619.2,414.3,4600.9,32.6,12.7,312.546
6,Tom Brady,TB,QB,29.1,49.9,2.1,0.0,0.0,0.0,3.2,672.9,445.1,4749.0,34.2,12.8,312.350
7,Kyler Murray,ARI,QB,98.0,467.1,4.5,0.0,0.0,0.0,3.0,568.7,384.9,4150.1,26.3,14.0,310.914
1,Jonathan Taylor,IND,RB,323.2,1614.2,13.8,42.8,364.6,1.7,2.7,0.0,0.0,0.0,0.0,0.0,306.880


Scraper not working. First initial errors were issues of not having lxml and html5lib installed. Intalling them helped it run BUT no tables were found came back as the output. 

In [ ]:
ADP_URL = 'https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php'

res = requests.get(ADP_URL)

soup = bs(res.content, 'html.parser')

table = soup.find('table', attr={
    'id': 'data'
})

adp_df = pd.read_html(str(table))

adp_df.head()

In [17]:
adp_df = pd.read_csv('/Users/brandynklee/Python_Projects/fantasy_footballl_22/data/ADP/half_ppr.csv')
adp_df = adp_df.loc[:, ['Player', 'Team', 'POS', 'Rank']]


#remove trailing numbers in POS column after each 2 letter position
pos_arr = ['RB', 'WR', 'TE', 'QB']

for pos in range(len(pos_arr)):
    mask = adp_df['POS'].str.startswith(pos_arr[pos][0], na = False)
    adp_df.loc[mask,['POS']] = pos_arr[pos]

adp_df.head(20)

,Player,Team,POS,Rank
0,Jonathan Taylor,IND,RB,1.0
1,Austin Ekeler,LAC,RB,2.0
2,Christian McCaffrey,CAR,RB,3.0
3,Cooper Kupp,LAR,WR,4.0
4,Derrick Henry,TEN,RB,5.0
5,Dalvin Cook,MIN,RB,6.0
6,Najee Harris,PIT,RB,7.0
7,Justin Jefferson,MIN,WR,8.0
8,Ja'Marr Chase,CIN,WR,9.0
9,Davante Adams,LV,WR,10.0


In [15]:
cutoff = 100
adp_cutoff_df = adp_df[:cutoff]


adp_cutoff_df.head(20)

,Player,Team,POS,Rank
0,Jonathan Taylor,IND,RB,1.0
1,Austin Ekeler,LAC,RB,2.0
2,Christian McCaffrey,CAR,RB,3.0
3,Cooper Kupp,LAR,WR,4.0
4,Derrick Henry,TEN,RB,5.0
5,Dalvin Cook,MIN,RB,6.0
6,Najee Harris,PIT,RB,7.0
7,Justin Jefferson,MIN,WR,8.0
8,Ja'Marr Chase,CIN,WR,9.0
9,Davante Adams,LV,WR,10.0
